# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 


# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [68]:
#setting file path to read in
file = os.path.join("..", "Output_Data", "cities.csv")

#read in csv
city_weather = pd.read_csv(file, delimiter=',')
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,68.00,52,75,12.75,AU,1609892714
1,College Station,30.6280,-96.3344,64.40,52,1,8.05,US,1609892615
2,Kapaa,22.0752,-159.3190,78.80,65,75,13.87,US,1609892716
3,Torbay,47.6666,-52.7314,37.99,100,90,17.22,CA,1609892718
4,Vila Franca do Campo,37.7167,-25.4333,55.40,71,75,13.87,PT,1609892719


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
gmaps.configure(api_key=gkey)

In [63]:
#set map locations and weight 
locations = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

#generate gmap 
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

#add heat layer to gmap 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
#narrowing down cities based on max temp, wind speed and cloudiness conditions
ideal_weather = city_weather.loc[(city_weather["Max Temp"] >= 70) & (city_weather["Max Temp"] <= 75) 
                                  & (city_weather["Wind Speed"] <10) & (city_weather["Cloudiness"] == 0)]
ideal_weather.dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,Gouré,13.9835,10.2704,72.54,19,0,8.12,NE,1609892819
129,Ondo,7.1000,4.8333,74.21,95,0,2.13,NG,1609892876
133,Det Udom,14.9060,105.0784,72.00,88,0,8.05,TH,1609892881
306,Bara,10.3744,10.7288,73.72,20,0,9.04,NG,1609893036
382,Bilma,18.6853,12.9164,70.00,22,0,4.68,NE,1609893198
522,Lafia,8.4833,8.5167,74.95,23,0,1.10,NG,1609893371
524,Valdivia,-39.8142,-73.2459,73.40,60,0,9.17,CL,1609893299
546,Duekoué,6.7472,-7.3496,73.60,91,0,3.24,CI,1609893402
560,Mocuba,-16.8392,36.9856,72.91,91,0,2.24,MZ,1609893420


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
#rename ideal weather df
hotel_df = pd.DataFrame(data=ideal_weather)
hotel_df["Hotel Name"] = ""
hotel_df.reindex()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
84,Gouré,13.9835,10.2704,72.54,19,0,8.12,NE,1609892819,
129,Ondo,7.1000,4.8333,74.21,95,0,2.13,NG,1609892876,
133,Det Udom,14.9060,105.0784,72.00,88,0,8.05,TH,1609892881,
306,Bara,10.3744,10.7288,73.72,20,0,9.04,NG,1609893036,
382,Bilma,18.6853,12.9164,70.00,22,0,4.68,NE,1609893198,
522,Lafia,8.4833,8.5167,74.95,23,0,1.10,NG,1609893371,
524,Valdivia,-39.8142,-73.2459,73.40,60,0,9.17,CL,1609893299,
546,Duekoué,6.7472,-7.3496,73.60,91,0,3.24,CI,1609893402,
560,Mocuba,-16.8392,36.9856,72.91,91,0,2.24,MZ,1609893420,


In [109]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

location = hotel_df[["Lat", "Lng"]].astype(float)

params = {"radius": 5000,
          "type": "lodging",
          "key": gkey}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = (f"{lat}, {lng}")
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    pprint(results)    
    
   
    
  
        
        
    
    
    


ValueError: could not convert string to float: 'Phama Lodge'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
